# Detect fDOM Plummeting Peaks

## Rules for plummeting peaks

Downward peak where: 
- Base width smaller than threshold 
- Prominence larger than a threshold 
- No corresponding peak in turbidity in a certain range 
- No other downward peaks in certain proximity (this would make the plummeting peak local fluctuation...) 

## Import Tools, Load and Preprocess Data

In [9]:
# Import libraries and data 
import scipy.io as sio
from sklearn.model_selection import TimeSeriesSplit
import copy
import pickle
import numpy as np
from scipy.signal import find_peaks
from os.path import dirname, join as pjoin
import datetime
import csv
import math
import sys
sys.path.insert(1,'../')
import Tools.data_processing as dp
import Tools.data_movement as dm 
from auxiliary_functions import get_candidates, detect_flat_plat, detect_stage_rises

fDOM_data = dm.read_in_preprocessed_timeseries('../Data/converted_data/julian_format/fDOM_raw_10.1.2011-9.4.2020.csv')
stage_data = dm.read_in_preprocessed_timeseries('../Data/converted_data/julian_format/stage_10.1.11-1.1.19.csv')
turb_data = dm.read_in_preprocessed_timeseries('../Data/converted_data/julian_format/turbidity_raw_10.1.2011_9.4.2020.csv')
stage_data = dp.align_stage_to_fDOM(fDOM_data, stage_data)

## Flip fDOM timeseries to detect plummeting peaks. Get candidate set for plummeting peaks
#### Flipping fDOM is surely inefficient and "peaks" could be detected in the unflipped timeseries

In [4]:
# Flip timeseries 
flipped_fDOM = dp.flip_timeseries(copy.deepcopy(fDOM_data))

# Get fDOM plummeting peak candidate set using scipy find_peaks()
prominence_range = [3,None] # peaks must have at least prominence 3
width_range = [None,10] # peaks cannot have a base width of more than 5
wlen = 100 
distance = 1 
rel_height =.6

# Get list of all peaks that could possibly be plummeting peaks
peaks, props = find_peaks(flipped_fDOM[:,1],
                          height = (None, None),
                          threshold = (None,None),
                          distance = distance,
                          prominence = prominence_range,
                          width = width_range,
                          wlen = wlen,
                          rel_height = rel_height)

# Form candidate set from returned information
cands = [[peak, math.floor(props['left_ips'][i]), math.ceil(props['right_ips'][i]),props['prominences'][i]] for i,peak in enumerate(peaks)]

/var/folders/hh/m6vvts714wdcv6sngjwj41zr0000gn/T/ipykernel_25502/31462634.py:12: PeakPropertyWarning: some peaks have a prominence of 0
  peaks, props = find_peaks(flipped_fDOM[:,1],


## Import ground truth labels

In [5]:
# Import and process ground truth
truth_fname = '../Data/labeled_data/ground_truths/fDOM/fDOM_Plummeting_Peaks/julian_time/fDOM_PLP_0k-300k.csv'

with open(truth_fname, 'r', newline = '') as f:
    
        reader = csv.reader(f, delimiter = ',')
        # truth entries in form: ['timestamp_of_peak', 'value_of_peak','label_of_peak','idx_of_peak']
        next(reader)
        truths = [[float(row[0]), float(row[1]), row[2], int(row[3])] for row in reader] 
        f.close()   

assert(len(truths) == len(cands))

## Detect turbidity peaks - necessary to distinquish between plummeting peak and interference

In [6]:
turb_peak_params = {'prom' : [6,None],
                    'width': [None, None],
                    'wlen' : 200,
                    'dist' : 1,
                    'rel_h': .6}

# Get fDOM and turb candiate peaks
turb_peaks, _ = get_candidates(turb_data, turb_peak_params)

## Define helper functions and set of training parameters, create training/testing splits

In [10]:
# Define Hyperparameter ranges and training parameters and helper function
iterations = 70000
num_splits = 5

basewidth_range = (1, 10)
prominence_range= (5, 20)

peak_proximity_bounds = (1,20)
turb_interference_bounds = (-10,10)


accumulated_test_metrics = {}

accumulated_test_results = {}

accumulated_best_params = {}

train_test_split_indices = TimeSeriesSplit(num_splits).split(cands)

def classify_candidate_peaks(peaks, params):
    
    def check_turb_interference():
        pass 
    def check_peak_proximity():
        pass 
    
    results = []
    for peak in peaks:
        prominence_condition = peak[3] >= params['min_prominence']
        basewidth_condition = abs(peak[1] - peak[2]) <= params['max_basewidth']
        turb_inteference_condition = check_turb_interference()
        peak_proximity_condition = check_peak_proximity()
        if prominence_condition and basewidth_condition and turb_interference_condition and peak_proximity_condition:
            results.append((peak[0], 'PLP'))
        else:
            results.append((peak[0], 'NPLP'))
    return results

def label_positives_negatives(predictions, truths):
    TP = TN = FP = FN = 0
    results = []
    
    for i in range(len(predictions)):
        prediction = predictions[i][1]
        truth = truths[i][2]
        
        if prediction == 'PLP':
            if truth == 'NPLP':
                FP +=1
                results.append(predictions[i].append('FP'))
            else: 
                TP +=1
                results.append(predictions[i].append('TP'))
        else:
            if truth == 'NPLP':
                TN +=1
                results.append(predictions[i].append('TN'))
            else: 
                FN +=1
                results.append(predictions[i].append('FN'))
        
    return (TP,TN,FP,FN,results)
    

## Nested Cross Validation Training Loop

In [ ]:
overall_start = datetime.datetime.now()

split = 1

for train_val_indices, test_indices in train_test_split_indices:
    X_train, y_train = [cands[i] for i in train_val_indices], [truths[i] for i in train_val_indices]
    X_test, y_test = [cands[i] for i in test_indices], [truths[i] for i in test_indices]
    
    max_fold_metric = 0 
    max_result = None 
    
    print("Split: ",split)
    
    split_start = datetime.datetime.now()
    
    for iteration in range(iterations):
        
        # Random grid search for hyperparams 
        params = {}
        
        params['max_basewidth'] = np.random.randint(basewidth_range[0], basewidth_range[1]+1)
        params['min_prominence'] = np.random.uniform(prominence_range[0], prominence_range[1])
        
        temp = np.random.randint(turb_interference_bounds[0], turb_interference_bounds[1])
        params['turb_interference_range'] = (temp, np.random.randint(temp, turb_interference_bounds[1]) + 2)
        
        params['peak_proximity_threshold']= np.random.randint(peak_proximity_bounds[0], peak_proximity_bounds[1])
      
        predictions = classify_candidate_peaks(X_train, params)
        
        TP,TN,FP,FN,results = label_positives_negatives(predictions, y_train)
        
        TPR = TP/(TP + FN)
        TNR = TN/(TN + FP)
        
        bal_acc = (TPR + TNR)/2
        
        f1_score = (2 * TP)/((2 * TP) + FP + FN)

        acc = f1_score
        if iteration and iteration % int(iterations/10) == 0: print(" {}/{} ".format(iteration, iterations), end = "")
        if acc > max_fold_metric: 
            max_fold_metric = acc
            max_result = copy.deepcopy(results)
            best_params = copy.deepcopy(params)
            
    # Test best parameters on testing data 
    test_predictions = classify_candidate_peaks(X_test, best_params)
    TP,TN,FP,FN,results = label_positives_negatives(test_predictions, y_test)
    
    TPR = TP/(TP + FN)
    TNR = TN/(TN + FP)
    
    bal_acc = (TPR + TNR)/2 
    f1_score = (2 * TP)/((2 * TP) + FP + FN)
    
    print('\nSplit: {}  F1: {:.4f} BA: {:.4f}  Params: {}  TP: {} TN: {} FP: {} FN: {}  Time: {}'.format(split, f1_score, bal_acc, best_params, TP, TN, FP, FN, datetime.datetime.now() - split_start))
    accumulated_test_metrics[split] = [f1_score, bal_acc] # Record test metrics of each split
    accumulated_test_results[split] = copy.deepcopy(results) # Record test results (FP,FN,TP,TN for each datapoint) for each split
    accumulated_best_params[split] = copy.deepcopy(best_params) # Record params uses in testing for each split
    
    split+=1

# Display Resuls 
mean_f1 = 0
mean_ba = 0 

for key in accumulated_test_metrics:
    metrics = accumulated_test_metrics[key]
    mean_f1+=metrics[0]
    mean_ba+=metrics[1]

print("Mean Test F1: ", mean_f1/len(accumulated_test_metrics))
print("Mean Test BA: ", mean_ba/len(accumulated_test_metrics))

print("Training time: ", datetime.datetime.now() - overall_start)

# Pickle params from last fold
with open('./Experiments/fDOM_PLP/best_params.pkl', 'wb') as pck_file:
    pickle.dump(accumulated_best_params[num_splits], pck_file)
    pck_file.close()
    
# Pickle results from last fold 
with open('./Experiments/fDOM_PLP/test_results.pkl', 'wb') as pck_file:
    pickle.dump(accumulated_test_results, pck_file)
    pck_file.close()

# Pickle results from last fold 
with open('./Experiments/fDOM_PLP/test_metrics.pkl', 'wb') as pck_file:
    pickle.dump(accumulated_test_metrics, pck_file)
    pck_file.close()

## Plot Results

In [3]:
turb_cand_params = {'prom' : [6,None],
                    'width': [None, None],
                    'wlen' : 200,
                    'dist' : 1,
                    'rel_h': .6}

# Get fDOM and turb candiate peaks
turb_peaks, turb_props = get_candidates(turb_data, turb_cand_params)

# Iterate through peaks and turn into short 3 point "events" by flagging the data point to either side of a peak
fDOM_events = []
fDOM_lb = []
fDOM_rb = []

for i,cand in enumerate(cands):
            fDOM_events.append(np.array((flipped_fDOM[cand[0]])))
            fDOM_lb.append(flipped_fDOM[math.floor(cand[1]),0])
            fDOM_rb.append(flipped_fDOM[math.ceil(cand[2]),0])
            
fDOM_lb = list(set(fDOM_lb))
fDOM_lb.sort()
fDOM_rb = list(set(fDOM_rb))
fDOM_rb.sort()

turb_events = []
turb_lb = []
turb_rb = []

for i,peak in enumerate(turb_peaks):
            turb_events.append(np.array((turb_data[peak])))
            turb_lb.append(turb_data[math.floor(turb_props['left_ips'][i]),0])
            turb_rb.append(turb_data[math.ceil(turb_props['right_ips'][i]),0])
            
turb_lb = list(set(turb_lb))
turb_lb.sort()
turb_rb = list(set(turb_rb))
turb_rb.sort()            

fDOM_merged = dp.merge_data(flipped_fDOM, fDOM_events, 'intf', '')
turb_merged = dp.merge_data(turb_data, turb_events, 't_opp', '')

fDOM_merged = dp.merge_additional_data(fDOM_merged, fDOM_lb, 'left_base')
fDOM_merged = dp.merge_additional_data(fDOM_merged, fDOM_rb, 'right_base')

turb_merged = dp.merge_additional_data(turb_merged, turb_lb, 'left_base')
turb_merged = dp.merge_additional_data(turb_merged, turb_rb, 'right_base')


stage_data_merged = dp.merge_data(stage_data, [], '','')

dm.write_data_to_trainset(fDOM_merged,
                          stage_data_merged,
                          turb_merged,
                          '../Data/temp_plotting/fDOM_plum_200k-300k.csv',
                          True,
                          True,
                          200000,
                          300000)